#  JACPTS implementation

In [2]:
import numpy as np
import math
import equadratures as eq
from numpy.polynomial import chebyshev
from scipy.special import jv, jn_zeros, comb


def csc(x):
    return 1 / np.sin(x)
def sec(x):
    return 1 / np.cos(x)

In [3]:
def asy(n, a, b):
    if n <= 20:  # Use only boundary formula:
        xbdy, wbdy, vbdy = asy2(n, a, b, np.ceil(n/2))
        xbdy2, wbdy2, vbdy2 = asy2(n, b, a, np.floor(n/2))
        x = np.concatenate([-xbdy2[::-1], xbdy])
        w = np.concatenate([wbdy2[::-1], wbdy])
        v = np.concatenate([vbdy2[::-1], vbdy])
        return x, w, v

    # Determine switch between interior and boundary regions:
    nbdy = 10
    bdyidx1 = np.arange(n - (nbdy - 1), n + 1)  # in Python, indexing starts from 0
    bdyidx2 = np.arange(nbdy - 1, -1, -1)

    # Interior formula:
    x, w, v = asy1(n, a, b, nbdy)

    # Boundary formula (right):
    xbdy, wbdy, vbdy = asy2(n, a, b, nbdy)
    x[bdyidx1] = xbdy
    w[bdyidx1] = wbdy
    v[bdyidx1] = vbdy

    # Boundary formula (left):
    if a != b:
        xbdy, wbdy, vbdy = asy2(n, b, a, nbdy)
    x[bdyidx2] = -xbdy
    w[bdyidx2] = wbdy
    v[bdyidx2] = vbdy

    return x, w, v

jacPtsTest = asy(10,1,3)
print(jacPtsTest)

NameError: name 'asy2' is not defined

In [ ]:
def asy1(n, a, b, nbdy):
    # Algorithm for computing nodes and weights in the interior.

    # Approximate roots via asymptotic formula: (Gatteschi and Pittaluga, 1985)
    K = (2 * (np.arange(n, 0, -1)) + a - 0.5) * np.pi / (2 * n + a + b + 1)
    tt = K + 1 / (2 * n + a + b + 1) ** 2 * ((0.25 - a ** 2) / np.tan(0.5 * K) - (0.25 - b ** 2) * np.tan(0.5 * K))

    # First half (x > 0):
    t = tt[tt <= np.pi / 2]
    mint = t[-nbdy]
    idx = np.arange(np.max([np.argwhere(t < mint)[0] - 1, 0]))

    dt = np.inf
    j = 0

    # Newton iteration
    while (np.linalg.norm(dt, np.inf) > np.sqrt(np.finfo(float).eps) / 100 and j < 10):
        vals, ders = feval_asy1(n, a, b, t, idx, 0)  # Evaluate
        dt = vals / ders  # Newton update
        t = t + dt  # Next iterate
        j = j + 1
        dt = dt[idx]

    vals, ders = feval_asy1(n, a, b, t, idx, 1)  # Once more for luck
    t = t + vals / ders  # Newton update.

    # Store:
    x = np.cos(t)
    w = 1 / ders ** 2
    v = np.sin(t) / ders

    # Second half (x < 0):
    a, b = b, a
    t = np.pi - tt[0:(n - len(x))]
    mint = t[nbdy]
    idx = np.arange(np.max([np.argwhere(t > mint)[0], 0]), len(t))

    dt = np.inf
    j = 0

    # Newton iteration
    while (np.linalg.norm(dt, np.inf) > np.sqrt(np.finfo(float).eps) / 100 and j < 10):
        vals, ders = feval_asy1(n, a, b, t, idx, 0)  # Evaluate
        dt = vals / ders  # Newton update
        t = t + dt  # Next iterate
        j = j + 1
        dt = dt[idx]

    vals, ders = feval_asy1(n, a, b, t, idx, 1)  # Once more for luck
    t = t + vals / ders  # Newton update

    # Store:
    x = np.concatenate([-np.cos(t), x])
    w = np.concatenate([1 / ders ** 2, w])
    v = np.concatenate([np.sin(t) / ders, v])

    return x, w, v


In [ ]:
def feval_asy1(n, a, b, t, idx, flag):
    M = 20
    onesT = ones(len(t))
    onesM = ones(M)
    MM = np.arange(0, M).reshape(-1, 1)
    alpha = (.5 * (2 * n + a + b + 1 + MM)) * onesT * t - .5 * (a + .5) * pi
    cosA = cos(alpha)
    sinA = sin(alpha)
    
    if flag:
        k = np.flip(np.arange(1, len(t) + 1)) if idx[0] == 1 else np.arange(1, len(t) + 1)
        ta = t.astype(np.float32)
        tb = t - ta
        hi = n * ta
        lo = n * tb + (a + b + 1) * .5 * t
        pia = 3.1415927410125732
        pib = -8.742278000372485e-08
        dh = (hi - (k - .25) * pia) + lo - .5 * a * pia - (k - .25 + .5 * a) * pib
        tmp = 0
        sgn = 1
        fact = 1
        DH = dh
        dh2 = dh * dh
        for j in range(21):
            dc = sgn * DH / fact
            tmp += dc
            sgn *= -1
            fact *= (2 * j + 3) * (2 * j + 2)
            DH *= dh2
            if np.linalg.norm(dc[idx], inf) < eps / 2000:
                break
        tmp[1::2] *= -1
        loc = np.argmax(abs(tmp[idx]))
        tmp = np.sign(cosA[0, idx[loc]] * tmp[idx[loc]]) * tmp
        cosA[0, :] = tmp

    sinT = onesM * sin(t)
    cosT = onesM * cos(t)
    cosA2 = cosA * cosT + sinA * sinT
    sinA2 = sinA * cosT - cosA * sinT

    sinT = np.concatenate([ones(1, len(t)), np.cumprod(onesM[1:] * (.5 * csc(.5 * t)))], axis=0)
    cosT = .5 * sec(.5 * t)

    j = np.arange(0, M - 1)
    vec = (.5 + a + j) * (.5 - a + j) / (j + 1) / (2 * n + a + b + j + 2)
    P1 = np.concatenate([[1], np.cumprod(vec)])
    P1[2::4] *= -1
    P1[3::4] *= -1
    P2 = eye(M)
    for l in range(M):
        j = np.arange(0, M - l - 1)
        vec = (.5 + b + j) * (.5 - b + j) / (j + 1) / (2 * n + a + b + j + l + 2)
        P2[l + 1:l + 1 + len(j), l] = np.cumprod(vec)
    PHI = np.repeat(P1, M, axis=0) * P2

    j = np.arange(0, M - 1)
    vec = (.5 + a + j) * (.5 - a + j) / (j + 1) / (2 * (n - 1) + a + b + j + 2)
    P1 = np.concatenate([[1], np.cumprod(vec)])
    P1[2::4] *= -1
    P1[3::4] *= -1
    P2 = eye(M)
    for l in range(M):
        j = np.arange(0, M - l - 1)
        vec = (.5 + b + j) * (.5 - b + j) / (j + 1) / (2 * (n - 1) + a + b + j + l + 2)
        P2[l + 1:l + 1 + len(j), l] = np.cumprod(vec)
    PHI2 = np.repeat(P1, M, axis=0) * P2

    S = 0
    S2 = 0
    SC = sinT
    for m in range(M):
        l = np.arange(0, m + 1, 2)
        phi = PHI[m, l]
        dS1 = phi @ (SC[l, :] * cosA[m, :])

        phi2 = PHI2[m, l]
        dS12 = phi2 @ (SC[l, :] * cosA2[m, :])

        l = np.arange(1, m + 1, 2)
        phi = PHI[m, l]
        dS2 = phi @ (SC[l, :] * sinA[m, :])

        phi2 = PHI2[m, l]
        dS22 = phi2 @ (SC[l, :] * sinA2[m, :])

        if m > 10 and np.linalg.norm(dS1[idx] + dS2[idx], inf) < eps / 100:
            break

        S += dS1 + dS2
        S2 += dS12 + dS22

        SC[:m + 1, :] *= cosT

    dsa = .5 * (a ** 2) / n
    dsb = .5 * (b ** 2) / n
    dsab = .25 * (a + b) ** 2 / n
    ds = dsa + dsb - dsab
    s = ds
    j = 1
    dsold = ds
    while (abs(ds / s) + dsold) > eps / 10:
        dsold = abs(ds / s)
        j += 1
        tmp = -(j - 1) / (j + 1) / n
        dsa *= tmp * a
        dsb *= tmp * b
        dsab *= .5 * tmp * (a + b)
        ds = dsa + dsb - dsab
        s += ds
    p2 = np.exp(s) * np.sqrt(2 * np.pi) * np.sqrt((n + a) * (n + b) / (2 * n + a + b)) / (2 * n + a + b + 1)
    g = np.array([1, 1 / 12, 1 / 288, -139 / 51840, -571 / 2488320, 163879 / 209018880,
                  5246819 / 75246796800, -534703531 / 902961561600,
                  -4483131259 / 86684309913600, 432261921612371 / 514904800886784000])

    def f(z):
        return np.sum(g * np.concatenate([np.array([1]), np.cumprod(np.ones(9) / z)]))

    C = p2 * (f(n + a) * f(n + b) / f(2 * n + a + b)) * 2 / np.pi
    C2 = C * (a + b + 2 * n) * (a + b + 1 + 2 * n) / (4 * (a + n) * (b + n))

    vals = C * S
    S2 = C2 * S2

    # Use relation for derivative:
    ders = (n * (a - b - (2 * n + a + b) * np.cos(t)) * vals + 2 * (n + a) * (n + b) * S2) / (2 * n + a + b) / np.sin(t)
    denom = 1 / np.real(np.sin(t / 2) ** (a + .5) * np.cos(t / 2) ** (b + .5))
    vals = vals * denom
    ders = ders * denom
    return vals, ders

In [ ]:
def asy2(n, a, b, npts):
    # Algorithm for computing nodes and weights near the boundary.

    # Use Newton iterations to find the first few Bessel roots:
    smallK = min(30, npts)
    jk = jn_zeros(a, min(npts, smallK))
    # Use asy formula for larger ones (See NIST 10.21.19, Olver 1974 p247)
    if npts > smallK:
        mu = 4*a**2
        a8 = 8*((np.arange(len(jk)+1,npts+1)+0.5*a-0.25)*np.pi)
        jk2 = 0.125*a8 - (mu-1)/a8 - 4*(mu-1)*(7*mu-31)/3/a8**3 - \
              32*(mu-1)*(83*mu**2-982*mu+3779)/15/a8**5 - \
              64*(mu-1)*(6949*mu**3-153855*mu**2+1585743*mu-6277237)/105/a8**7
        jk = np.concatenate([jk, jk2])
    jk = np.real(jk[0:npts])

    # Approximate roots via asymptotic formula: (see Olver 1974, NIST, 18.16.8)
    rho = n + 0.5*(a + b + 1) 
    phik = jk/rho
    t = phik + ((a**2-0.25)*(1-phik/np.tan(phik))/(2*phik) - \
        0.25*(a**2-b**2)*np.tan(0.5*phik))/rho**2

    # Compute higher terms:
    tB1, A2, tB2, A3 = asy2_higherterms(a, b, t, n)

    dt = np.inf
    j = 0
    # Newton iteration:
    while np.linalg.norm(dt, np.inf) > np.sqrt(np.finfo(float).eps)/200 and j < 10:
        vals, ders = feval_asy2(n, t, 1) # Evaluate via asymptotic formula.
        dt = vals/ders                   # Newton update.
        t = t + dt                       # Next iterate.
        j += 1
    vals, ders = feval_asy2(n, t, 1)     # Evaluate via asymptotic formula.
    dt = vals/ders                       # Newton update
    t = t + dt

    # flip:
    t = t[npts-1::-1]
    ders = ders[npts-1::-1]

    # Revert to x-space:
    x = np.cos(t)      
    w = (1./ders**2).T   
    v = np.sin(t)/ders

    return x, w, v

## VERIFIED FUNCTIONS

In [4]:
def chebpts(n): #====================================VERIFIED====================================
    """
    Returns the Chebyshev points, quadrature weights, barycentric weights, and angles in [-1,1].
    
    chebtech2
    """
    if n == 0:  # Special case (no points)
        x = np.array([])
        w = np.array([])
        v = np.array([])
        t = np.array([])
        
    elif n == 1:  # Special case (single point)
        x = np.array([0])
        w = np.array([2])
        v = np.array([1])
        t = np.array([np.pi / 2])
        
    else:  # General case
        # Chebyshev points:
        m = n - 1
        x = np.sin(np.pi * np.arange(-m, m + 1, 2) / (2 * m))

        # Quadrature weights:            
        w = quadwts(n) if n > 1 else None

        # Barycentric weights:            
        v = barywts(n) if n > 2 else None

        # Angles:
        t = angles(n) if n > 3 else None
        
    return x, w, v, t



def quadwts(n): #====================================VERIFIED====================================
    """
    
    chebtech2
    """
    if n == 0:
        w = []
    elif n==1:
        w = 2
    else:
        c = 2 / np.concatenate(([1], 1 - np.arange(2, n, 2)**2))
        c = np.concatenate((c, c[np.floor(n/2).astype(int):0:-1]))
        w = np.real(np.fft.ifft(c))
        w = np.concatenate((w, [w[0]/2]))
        w[0] = w[0] / 2
    return w




def barywts(n): #====================================VERIFIED====================================
    """
    Returns the barycentric weights for polynomial interpolation on a Chebyshev grid of the 2nd kind.
    
    chebtech2
    """
    if n == 0:  # Special case (no points)
        v = np.array([])
    elif n == 1:  # Special case (single point)
        v = np.array([1])
    else:  # General case
        v = np.concatenate((np.ones(n - 1), [0.5]))
        v[-2::-2] = -1
        v[0] *= 0.5
    return v


def angles(n): #====================================VERIFIED====================================
    """
    Returns the angles of the Chebyshev points of 2nd kind in [-1, 1].
    
    chebtech2
    """
    if n == 0:
        return np.array([])
    elif n == 1:
        return np.array([np.pi / 2])
    else:
        m = n - 1
        return np.array([i * np.pi / m for i in range(m, -1, -1)])
        


In [5]:
def bary_weights(x): #====================================VERIFIED====================================
    """
    
    baryWeights.m
    """
    
    # Convert x to numpy array
    x = np.array(x)
    
    # Ensure input is a 1-D array
    if x.ndim != 1:
        raise ValueError("Input must be a vector")
    
    n = x.size

    # Capacity
    C = 1
    if np.isreal(x).all():
        C = 4 / (np.max(x) - np.min(x))   # Capacity of interval
    
    # Compute the weights
    if n < 2001:
        # For small n using matrices is faster
        V = C * (x[:, np.newaxis] - x)
        np.fill_diagonal(V, 1)
        VV = np.exp(np.sum(np.log(np.abs(V)), axis=1))
        w = 1 / (np.prod(np.sign(V), axis=1) * VV)
    else:
        # For large n use a loop
        w = np.ones(n)
        for j in range(n):
            v = C * (x[j] - x)
            v[j] = 1
            vv = np.exp(np.sum(np.log(np.abs(v))))
            w[j] = 1 / (np.prod(np.sign(v)) * vv)
    
    # Scaling
    w = w / np.linalg.norm(w, np.inf)
    
    if w.size % 2 == 0:
        return -w
    else:
        return w

In [6]:
def baryDiffMat(x, w=None, k=1, t=None): #====================================VERIFIED====================================
    #Only verified for one input, test w, k, and t if further problems
    
    """
    BARYDIFFMAT  Barycentric differentiation matrix.
    D = BARYDIFFMAT(N, X) is the matrix that maps function values at the points
    X to values of the derivative of the interpolating polynomial at those
    points.
    
    D = BARYDIFFMAT(X, W) is the same, but here the derivative is of the
    barycentric interpolant defined by the points X and the weights W.
    
    D = BARYDIFFMAT(X, W, K) is the same, but for the Kth derivative.
    
    D = BARYDIFFMAT(X, W, K, T) but accepts T = ACOS(X), which allows more
    accurate computation of pairwise differences of X in some cases. (See [4]).
    
    chebcolloc -> baryDiffMat.m
    """
    N = len(x)
    
    ## Parse inputs and check for trivial cases:
    if (N == 0):
        return []
    elif (N == 1):
        return np.array([0])
    if (w is None):
        w = bary_weights(x)
        
    if len(x) != len(w):
        raise ValueError('length of x must equal length of w.')
        
    ## Construct Dx and Dw:
    ii = np.diag_indices(N)                              # Indices of diagonal.
    if t is not None:
        # Trig identity as described in [4]:
        t = np.flip(t)
        Dx = 2*np.sin(np.add.outer(t/2, t/2))*np.sin(np.subtract.outer(t/2, t/2)) 
    else:
        Dx = np.subtract.outer(x, x)                     # All pairwise differences.
    
    # Flipping trick in [4]:
    DxRot = np.rot90(Dx, 2)
    idxTo = np.triu(np.ones((N,N)), 1).astype(bool)
    Dx[idxTo] = -DxRot[idxTo]
    
    Dx[ii] = 1                                          # Add identity.
    Dxi = 1. / Dx                                       # Reciprocal.
    Dw = w / w[:, np.newaxis]                         # Pairwise divisions.
    Dw[ii] = 0                                          # Subtract identity.
    
    ## k = 1
    D = Dw * Dxi
    D[ii] = 0; D[ii] = -D.sum(axis=1)                   # Negative sum trick.
    
    # Forcing symmetry for even N:
    N_half_floor = N // 2
    D.flat[N_half_floor*(N+1):N**2:N+1] = -D.flat[N_half_floor*(N+1):-1:-(N+1)]
    
    if (k == 1):
        return D
    
    ## k = 2
    D = 2 * D * (np.tile(D[ii], (N, 1)).transpose() - Dxi)
    D[ii] = 0; D[ii] = -D.sum(axis=1)                   # Negative sum trick.
    
    ## k = 3...
    for n in range(3, k+1):
        D = n*Dxi * ( Dw*np.tile(D[ii], (N, 1)).transpose() - D )
        D[ii] = 0; D[ii] = -D.sum(axis=1)               # Negative sum trick.
        print("D: ", D)
    
    return D

In [7]:
def diffmat(N, k=1): #====================================VERIFIED====================================
    """
    Chebyshev differentiation matrix.
    
    chebcolloc2
    """

    # Get Chebyshev points of the first kind
    x = chebpts(N)
    
    # Get Barycentric weights
    w = barywts(N)
    
    # Get acos(x)
    t = angles(N)

    # Construct matrix
    D = baryDiffMat(x[0], w, k, t)
    
    return D

In [56]:
def bary(x, fvals): #====================================VERIFIED====================================
    """
    Barycentric interpolation on a 2nd-kind Chebyshev grid.
    Evaluates G(X) using the barycentric interpolation formula, where F is the polynomial interpolant
    on a 2nd-kind Chebyshev grid to the values stored in the columns of FVALS. X should be a column vector.
    """

    # Parse inputs:
    n = fvals.shape[0]
    fvals = np.matrix(fvals).transpose()

    # Chebyshev nodes and barycentric weights:
    xk, xk1, xk2, xk3 = chebpts(n)
    vk = barywts(n)

    # Call BARY:
    fx = bary1(x, fvals, xk, vk)

    return fx


In [ ]:
def bary1(x, fvals, xk=None, vk=None): #====================================VERIFIED====================================
    """
    Barycentric interpolation formula.

    Uses the 2nd form barycentric formula with weights VK
    to evaluate an interpolant of the data {XK, FVALS[:,k]} at the points X.
    Note that XK and VK should be column vectors, and FVALS, XK, and VK should
    have the same length.
    """
    
    # Parse inputs:
    n, m = fvals.shape
    sizex = x.shape
    ndimsx = x.ndim

    if m > 1 and ndimsx > 2:
        raise ValueError("BARY does not support evaluation of vectors of polynomials at inputs with more than two dimensions.")

    # Default to Chebyshev nodes and barycentric weights:
    if xk is None:
        xk = chebpts(n)
    if vk is None:
        vk = barywts(n)

    if not np.all(sizex):
        fx = x
        return fx

    # Check that input is a column vector:
    if ndimsx > 2 or sizex[1] > 1:
        x = x.flatten(order = 'F')

    # The function is a constant.
    if n == 1:
        fx = np.repeat(fvals, len(x), axis=0)
        return fx

    # The function is NaN.
    if np.any(np.isnan(fvals)):
        fx = np.full((len(x), m), np.nan)
        return fx

    # The main loop:
    if x.size < 4*xk.size:  # Loop over evaluation points
        # Initialise return value:
        fx = np.zeros((x.size, m))

        # Loop:
        vk = vk.reshape(-1,1)
        for j in range(x.size):
            xx = vk / (x[j, 0] - xk)
            fx[j, :] = np.dot(xx.T, fvals) / np.sum(xx)
            
    else:                   # Loop over barycentric nodes
        # Initialise:
        num = np.zeros((x.size, m), dtype=np.complex128)
        denom = np.zeros((x.size, m), dtype=np.complex128)

        # Loop:
        for j in range(xk.size):
            tmp = vk[j] / (x - xk[j])
            num += np.multiply(tmp[:, np.newaxis], fvals[j, :])
            tmp = tmp.reshape(n**2, 1)
            denom += tmp
        fx = num / denom

    # Try to clean up NaNs:
    for k in np.where(np.isnan(fx[:, 0]))[0]:
        index = np.where(x[k] == xk)[0]  # Find the corresponding node
        if index.size:
            fx[k, :] = fvals[index[0], :]  # Set entry/entries to the correct value

    # Reshape the output if possible:
    if m == 1 and (ndimsx > 2 or sizex[1] > 1):
        fx = fx.reshape(sizex)
    elif m > 1 and (ndimsx == 2 or sizex[1] > 1):
        fx = fx.reshape(sizex[0], m * x.size // sizex[0])

    return fx

In [120]:
def cumsummat(N): #====================================VERIFIED====================================
    """
    Chebyshev integration matrix.
    Returns the matrix that maps function values at N Chebyshev points to values of the integral 
    of the interpolating polynomial at those points, with the convention that the first value is zero.
    
    chebcolloc2.cumsummat
    """

    N = N - 1

    if N == 0:
        return np.array([1])

    # Matrix mapping coeffs -> values.
    T = coeffs2vals(np.eye(N+1))

    # Matrix mapping values -> coeffs.
    Tinv = vals2coeffs(np.eye(N+1))

    # Matrix mapping coeffs -> integral coeffs. Note that the highest order term is truncated.
    k = np.arange(1, N+1)
    k2 = 2 * (k - 1)
    k2[0] = 1  # avoid divide by zero
    B = np.diag(1/(2*k), -1) - np.diag(1/k2, 1)
    v = np.ones(N)
    v[1::2] = -1
    B[0, :] = np.sum(np.diag(v) @ B[1:N+1, :], axis=0)
    B[:, 0] = 2 * B[:, 0]

    Q = T @ B @ Tinv

    # Make exact:
    Q[0, :] = 0

    return Q

def coeffs2vals(coeffs): #====================================VERIFIED====================================
    """
    Convert Chebyshev coefficients to values at Chebyshev points.
    
    chebtech2
    """
    n = coeffs.shape[0]
    # Trivial case (constant or empty)
    if n <= 1:
        return coeffs

    # Check for symmetry
    isEven = np.max(np.abs(coeffs[1::2, :]), axis=0) == 0
    isOdd = np.max(np.abs(coeffs[::2, :]), axis=0) == 0

    # Scale them by 1/2
    coeffs[1:n-1, :] = coeffs[1:n-1, :] / 2

    # Mirror the coefficients (to fake a DCT using an FFT)
    tmp = np.vstack((coeffs, coeffs[n-2:0:-1, :]))

    if np.isrealobj(coeffs):
        # Real-valued case
        values = np.real(np.fft.fft(tmp, axis=0))
    elif np.isrealobj(1j * coeffs):
        # Imaginary-valued case
        values = 1j * np.real(np.fft.fft(np.imag(tmp), axis=0))
    else:
        # General case
        values = np.fft.fft(tmp, axis=0)

    # Flip and truncate
    values = values[n-1::-1, :]

    # Enforce symmetry
    values[:, isEven] = (values[:, isEven] + np.flipud(values[:, isEven])) / 2
    values[:, isOdd] = (values[:, isOdd] - np.flipud(values[:, isOdd])) / 2

    return values

def vals2coeffs(values): #====================================VERIFIED====================================
    """
    Convert values at Chebyshev points to Chebyshev coefficients.
    
    chebtech2
    """
    
    n = values.shape[0]
    # Trivial case (constant)
    if n <= 1:
        return values

    # Check for symmetry
    isEven = np.max(np.abs(values - np.flipud(values)), axis=0) == 0
    isOdd = np.max(np.abs(values + np.flipud(values)), axis=0) == 0

    # Mirror the values (to fake a DCT using an FFT)
    tmp = np.vstack((values[n-1:0:-1, :], values[0:n-1, :]))

    if np.isrealobj(values):
        # Real-valued case
        coeffs = np.fft.ifft(tmp, axis=0)
        coeffs = np.real(coeffs)
    elif np.isrealobj(1j * values):
        # Imaginary-valued case
        coeffs = np.fft.ifft(np.imag(tmp), axis=0)
        coeffs = 1j * np.real(coeffs)
    else:
        # General case
        coeffs = np.fft.ifft(tmp, axis=0)

    # Truncate
    coeffs = coeffs[:n, :]

    # Scale the interior coefficients
    coeffs[1:n-1, :] = 2 * coeffs[1:n-1, :]

    # Adjust coefficients for symmetry
    coeffs[1::2, isEven] = 0
    coeffs[::2, isOdd] = 0

    return coeffs



def besselTaylor(t, z, a): #==================VERIFIED==============
    """
    Accurate evaluation of Bessel function J_A for asy2.
    Evaluates J_A(Z+T) by a Taylor series expansion about Z.
    """
    
    npts = np.size(t)
    kmax = min(math.ceil(abs(math.log(np.finfo(float).eps)/math.log(np.linalg.norm(t, np.inf)))), 30)
    H = np.power(t, np.arange(0,kmax+1)).T

    # Compute coeffs in Taylor expansions about z (See NIST 10.6.7)
    nu, JK = np.meshgrid(np.arange(-kmax, kmax+1), z)
    Bjk = jv(a + nu, JK)

    nck = np.zeros((int(1.25*kmax), int(1.25*kmax)))
    for i in range(int(1.25*kmax)):
        for j in range(i+1):
            nck[i, j] = comb(i, j, exact=True)

    nck = np.delete(nck, (0), axis=0)
    AA = np.concatenate((Bjk[:,kmax].reshape(-1,1), np.zeros((npts, kmax))), axis=1)
    fact = 1

    for k in range(1, kmax+1):
        sgn = 1
        for l in range(0, k+1):
            AA[:,k] = AA[:,k] + sgn*nck[k-1,l] * Bjk[:,kmax+2*l-k]
            sgn = -sgn
        fact *= k
        AA[:,k] = AA[:,k] / (2**k*fact)
    
    # Evaluate Taylor series:
    Ja = np.zeros((npts, 1))
    for k in range(0, npts):
        Ja[k] = np.matmul(AA[k,:], H[:,k])
    return Ja

In [149]:
def asy2_1(n, a, b, npts): #=======================VERIFIED=====================
    # Algorithm for computing nodes and weights near the boundary.

    # Use Newton iterations to find the first few Bessel roots:
    smallK = min(30, npts)
    jk = jn_zeros(a, min(npts, smallK))  
    # Use asy formula for larger ones (See NIST 10.21.19, Olver 1974 p247)
    
    
    #Need to check if statement
    if npts > smallK:
        mu = 4*a**2
        a8 = 8*((np.arange(len(jk)+1,npts+1)+0.5*a-0.25)*np.pi)
        jk2 = 0.125*a8 - (mu-1)/a8 - 4*(mu-1)*(7*mu-31)/3/a8**3 - \
              32*(mu-1)*(83*mu**2-982*mu+3779)/15/a8**5 - \
              64*(mu-1)*(6949*mu**3-153855*mu**2+1585743*mu-6277237)/105/a8**7
        jk = np.concatenate([jk, jk2])

        
    jk = np.real(jk[0:npts])

    # Approximate roots via asymptotic formula: (see Olver 1974, NIST, 18.16.8)
    rho = n + 0.5*(a + b + 1) 
    phik = jk/rho
    t = phik + ((a**2-0.25)*(1-phik/np.tan(phik))/(2*phik) - \
        0.25*(a**2-b**2)*np.tan(0.5*phik))/rho**2
    
    
    
    # Compute higher terms:
#     tB1, A2, tB2, A3, v = asy2_higherterms(a, b, t, n)

    dt = np.inf
    j = 0
    # Newton iteration:
    while np.all(np.abs(dt) > np.sqrt(np.finfo(float).eps) / 200) and j < 10:
        t=t.reshape(-1,1)
        vals, ders = feval_asy2(n, t, 1, a,b, rho) # Evaluate via asymptotic formula.
        dt = vals/ders                   # Newton update.
        t = t + dt                       # Next iterate.
        j += 1
    vals, ders = feval_asy2(n, t, 1, a, b, rho)     # Evaluate via asymptotic formula.
    dt = vals/ders                       # Newton update
    t = t + dt
    # flip:
    t = t[npts-1::-1]
    ders = ders[npts-1::-1]

    # Revert to x-space:
    x = np.cos(t)      
    w = (1./ders**2).T   
    v = np.sin(t)/ders
    
    
    return x, w, v

# SEMI VERIFIED (EXPECT BUGS)

In [41]:
def asy2_higherterms(alph, bet, theta, n):
    # These constants are more useful than alph and bet:
    A = (0.25 - alph**2)
    B = (0.25 - bet**2)

    # For now, just work on half of the domain:
    c = max(max(theta), .5)
    if n < 30:
        N = np.ceil(40 - n)
    elif n >= 30 and c > np.pi/2 - .5:
        N = 15
    else:
        N = 10

    Nm1 = N - 1

    # Scaled 2nd-kind Chebyshev points and barycentric weights:
    t = .5 * c * (np.sin(np.pi * np.arange(-Nm1, Nm1 + 1, 2) / (2*Nm1)) + 1)
    v = np.concatenate(([0.5], np.ones(int(Nm1))))
    v[1::2] = -1
    v[-1] = .5 * v[-1]

    # The g's:
    g = A * (1/np.tan(t/2) - 2/t) - B * np.tan(t/2)
    gp = A * (2/t**2 - .5/np.sin(t/2)**2) - .5 * (.25 - bet**2) / np.cos(t/2)**2
    gpp = A * (-4/t**3 + .25 * np.sin(t) / np.sin(t/2)**4) - 4 * B * np.sin(t/2)**4 / np.sin(t)**3
    g[0] = 0; gp[0] = -A/6 - .5 * B; gpp[0] = 0
    

    # B0:
    B0 = .25 * g / t
    B0p = .25 * (gp/t - g/t**2)
    B0[0] = .25 * (-A/6 - .5 * B)
    B0p[0] = 0

    # A1:
    A10 = alph * (A + 3 * B) / 24
    A1 = .125 * gp - (1 + 2 * alph) / 2 * B0 - g**2 / 32 - A10
    A1p = .125 * gpp - (1 + 2 * alph) / 2 * B0p - gp * g / 16
    A1p_t = A1p / t
    A1p_t[0] = -A/720 - A**2/576 - A*B/96 - B**2/64 - B/48 + alph * (A/720 + B/48)
    
    # Make f accurately: (Taylor series approx for small t)
    fcos = B / (2 * np.cos(t/2))**2
    f = -A * (1/12 + t**2/240 + t**4/6048 + t**6/172800 + t**8/5322240 + \
        691 * t**10/118879488000 + t**12/5748019200 + \
        3617 * t**14/711374856192000 + 43867 * t**16/300534953951232000)
    idx = t > .5
    ti = t[idx]
    f[idx] = A * (1/ti**2 - 1/(2 * np.sin(ti/2))**2)
    f = f - fcos

    # Integrals for B1: (Note that N isn't large, so we don't need to be fancy).
    N = int(N)
    C = cumsummat(N) * (.5 * c)
    D = diffmat(N) * (2 / c)
    I = (C @ A1p_t)
    J = (C @ (f * A1))

    # B1:
    tB1 = -.5 * A1p - (.5 + alph) * I + .5 * J
    tB1[0] = 0
    B1 = tB1 / t
    B1[0] = A/720 + A**2/576 + A*B/96 + B**2/64 + B/48 + \
        alph * (A**2/576 + B**2/64 + A*B/96) - alph**2 * (A/720 + B/48)
    # A2:
    K = C @ (f * tB1)
    
    tB1 = tB1.reshape(-1, 1)
    B1 = B1.reshape(-1, 1)
    K = K.reshape(-1, 1)
    A2 = .5 * (D @ tB1) - (.5 + alph) * B1 - .5 * K
    A2 = A2 - A2[0]

    # Skip to output if nargout < 3
#     if 'A2p' not in locals():
#         tB1 = bary(tB1, t, v)
#         A2 = bary(A2, t, v)
#         return tB1, A2

    
    A2p = D @ A2
    A2p = A2p - A2p[0]
    A2p_t = A2p.flatten() / t.flatten()
    A2p_t = A2p_t.reshape(-1,1)

    # Extrapolate point at t = 0
    w = np.pi/2 - t[1:]
    w[1::2] = -w[1::2]
    w[-1] = .5*w[-1]
    w = w.reshape(-1, 1)
    A2p_t[0] = np.sum(w.flatten() * A2p_t[1:].flatten()) / np.sum(w)

    # B2:
    f = f.reshape(-1,1)
    tB2 = -0.5 * A2p - (0.5 + alph) * (C @ A2p_t) + 0.5 * (C @ (f * A2).flatten()).reshape(-1,1)
    B2 = tB2.flatten() / t.flatten();
    B2 = B2.reshape(-1,1)
    # Extrapolate point at t = 0:
    B2[0] = np.sum(w.flatten() * B2[1:].flatten()) / np.sum(w)

    # A3:
    K = C @ (f * tB2)
    A3 = .5 * (D @ tB2) - (.5 + alph) * B2 - .5 * K
    A3 = A3 - A3[0]

    t = t.reshape(-1,1)
    tB1x = lambda theta: bary1(theta, tB1, xk=t, vk=v)
    A2x = lambda theta: bary1(theta, A2, xk=t, vk=v)
    tB2x = lambda theta: bary1(theta, tB2, xk=t, vk=v)
    A3x = lambda theta: bary1(theta, A3, xk=t, vk=v)

    return tB1x, A2x, tB2x, A3x, v

In [122]:
def feval_asy2(n, t, flag, a, b, rho):
    #=================VALS IS SLIGHTLY OFF, DERS IS GOOD===============
    # Evaluate the boundary asymptotic formula at x = cos(t).

    
    #Evaluate higher terms
    tB1, A2, tB2, A3, v = asy2_higherterms(a, b, t, n)
    
    
    # Useful constants:
    rho2 = n + 0.5*(a + b - 1)
    A = (0.25 - a**2)
    B = (0.25 - b**2)

    # Evaluate the Bessel functions:
    Ja = jv(a, rho*t)
    Jb = jv(a + 1, rho*t)
    Jbb = jv(a + 1, rho2*t)
    if not flag:
        Jab = jv(a, rho2*t)
    else:
        # In the final step, perform accurate evaluation
        Jab = besselTaylor(-t, rho*t, a)
    # Evaluate functions for recurrsive definition of coefficients:
    gt = A*(1/np.tan(t/2)-(2/t)) - B*np.tan(t/2)
    gtdx = A*(2/t**2-0.5/(np.sin(t/2))**2) - 0.5*B/(np.cos(t/2))**2
    tB0 = 0.25*gt
    A10 = a*(A+3*B)/24
    A1 = gtdx/8 - (1+2*a)/8*gt/t - gt**2/32 - A10
    # Higher terms:
    tB1t = tB1(t)
    A2t = A2(t)
    # VALS:
    vals = Ja + Jb*tB0/rho + Ja*A1/rho**2 + Jb*tB1t/rho**3 + Ja*A2t/rho**4
    # DERS:
    vals2 = Jab + Jbb*tB0/rho2 + Jab*A1/rho2**2 + Jbb*tB1t/rho2**3 + Jab*A2t/rho2**4

    # Higher terms (not needed for n > 1000).
    #====================TB2T IS MILDLY OFF
    tB2t = tB2(t)
    #====================A3T IS DECENTLY OFF
    A3t = A3(t)
    vals = vals + Jb*tB2t/rho**5 + Ja*A3t/rho**6
    vals2 = vals2 + Jbb*tB2t/rho2**5 + Jab*A3t/rho2**6

    # Constant out the front (Computed accurately!)
    ds = 0.5*(a**2)/n
    s = ds
    jj = 1
    while np.abs(ds/s) > np.finfo(float).eps/10:
        jj += 1
        ds = -(jj-1)/(jj+1)/n*(ds*a)
        s = s + ds
    p2 = np.exp(s) * np.sqrt((n+a) / n) * (n / rho) ** a
    g = np.array([1, 1/12, 1/288, -139/51840, -571/2488320, 163879/209018880,
                 5246819/75246796800, -534703531/902961561600,
                 -4483131259/86684309913600, 432261921612371/514904800886784000])

    # Define the function f(z)
    def f(z):
        return np.sum(g * np.concatenate(([1], np.cumprod(np.ones(9) / z))))

    # Calculate C
    C = p2 * (f(n + a) / f(n)) / np.sqrt(2)

    # Scaling:
    valstmp = C * vals
    denom = np.sin(t/2) ** (a + 0.5) * np.cos(t/2) ** (b + 0.5)
    vals = np.sqrt(t) * valstmp / denom

    # Relation for derivative:
    C2 = C * n / (n + a) * (rho / rho2) ** a
    ders = (n * (a - b - (2 * n + a + b) * np.cos(t)) * valstmp + 2 * (n + a) * (n + b) * C2 * vals2) / (2 * n + a + b)
    ders = ders * (np.sqrt(t) / (denom * np.sin(t)))
    
    
    return vals, ders

## TESTING

In [11]:
#=====CHECK ANGLES=====
# CHEB:
#     x =
#     3.1416
#     2.7925
#     2.4435
#     2.0944
#     1.7453
#     1.3963
#     1.0472
#     0.6981
#     0.3491
#          0

anglesCheck = angles(10)
print(anglesCheck)

[3.14159265 2.7925268  2.44346095 2.0943951  1.74532925 1.3962634
 1.04719755 0.6981317  0.34906585 0.        ]


In [12]:
#=====CHECK barywts=====
# chebweights =

#    -0.5000
#     1.0000
#    -1.0000
#     1.0000
#    -1.0000
#     1.0000
#    -1.0000
#     1.0000
#    -1.0000
#     0.5000

barywtsCheck = barywts(10)
print(barywtsCheck)
    

[-0.5  1.  -1.   1.  -1.   1.  -1.   1.  -1.   0.5]


In [13]:
#=====CHECK QUAD WTS======
# quadWeights =

#     0.0123    0.1166    0.2253    0.3019    0.3439    0.3439    0.3019    0.2253    0.1166    0.0123\
quadCheck = quadwts(10)
print(quadCheck)

[0.01234568 0.11656746 0.22528432 0.30194004 0.34386251 0.34386251
 0.30194004 0.22528432 0.11656746 0.01234568]


In [14]:
#=====CHECK CHEB PTS=====

# [chebptsCheck1, chebptsCheck2, chebptsCheck3, chebptsCheck4] = chebtech2.chebpts(10)
# chebptsCheck1 =

#    -1.0000
#    -0.9397
#    -0.7660
#    -0.5000
#    -0.1736
#     0.1736
#     0.5000
#     0.7660
#     0.9397
#     1.0000


# chebptsCheck2 =

#     0.0123    0.1166    0.2253    0.3019    0.3439    0.3439    0.3019    0.2253     0.1166    0.0123


# chebptsCheck3 =

#    -0.5000
#     1.0000
#    -1.0000
#     1.0000
#    -1.0000
#     1.0000
#    -1.0000
#     1.0000
#    -1.0000
#     0.5000


# chebptsCheck4 =

#     3.1416
#     2.7925
#     2.4435
#     2.0944
#     1.7453
#     1.3963
#     1.0472
#     0.6981
#     0.3491
#          0

chebPtsCheck1, chebPtsCheck2, chebPtsCheck3, chebPtsCheck4, = chebpts(10)
print("1: ", chebPtsCheck1)
print()
print("2: ", chebPtsCheck2)
print()
print("3: ", chebPtsCheck3)
print()
print("4: ", chebPtsCheck4)

1:  [-1.         -0.93969262 -0.76604444 -0.5        -0.17364818  0.17364818
  0.5         0.76604444  0.93969262  1.        ]

2:  [0.01234568 0.11656746 0.22528432 0.30194004 0.34386251 0.34386251
 0.30194004 0.22528432 0.11656746 0.01234568]

3:  [-0.5  1.  -1.   1.  -1.   1.  -1.   1.  -1.   0.5]

4:  [3.14159265 2.7925268  2.44346095 2.0943951  1.74532925 1.3962634
 1.04719755 0.6981317  0.34906585 0.        ]


In [15]:
#=====Check bary_weights=====
# baryWeightscheck =

#     0.0079
#    -0.0714
#     0.2857
#    -0.6667
#     1.0000
#    -1.0000
#     0.6667
#    -0.2857
#     0.0714
#    -0.0079

baryWeightsCheck = bary_weights([1,2,3,4,5,6,7,8,9,10])
print(baryWeightsCheck)

[ 0.00793651 -0.07142857  0.28571429 -0.66666667  1.         -1.
  0.66666667 -0.28571429  0.07142857 -0.00793651]


In [16]:
#=====CHECK BARY DIFF MAT=====

# baryDiffMatCheck = chebcolloc.baryDiffMat([1,2,3])
# baryDiffMatCheck =

#     1.5000   -2.0000    0.5000
#     0.5000         0   -0.5000
#    -0.5000    2.0000   -1.5000

baryDiffMatCheck = baryDiffMat([1,2,3])
print(baryDiffMatCheck)

[[-1.5  2.  -0.5]
 [-0.5 -0.   0.5]
 [ 0.5 -2.   1.5]]


In [17]:
#=====CHECK DIFF MAT=====
# diffMatCheck = chebcolloc2.diffmat(4,2)
# diffMatCheck =
#     5.3333   -9.3333    6.6667   -2.6667
#     3.3333   -5.3333    2.6667   -0.6667
#    -0.6667    2.6667   -5.3333    3.3333
#    -2.6667    6.6667   -9.3333    5.3333

diffMatCheck = diffmat(4,2)
print(diffMatCheck)

[[ 5.33333333 -9.33333333  6.66666667 -2.66666667]
 [ 3.33333333 -5.33333333  2.66666667 -0.66666667]
 [-0.66666667  2.66666667 -5.33333333  3.33333333]
 [-2.66666667  6.66666667 -9.33333333  5.33333333]]


In [18]:
#=====Check bary=====

# xcheb = chebtech2.chebpts(4)
# fx = 1./( 1 + 25*xcheb.^2 )
# xx = linspace(-1, 1, 4);
# [xx, yy] = meshgrid(xx, xx)
# ff = bary(xx + 1i*yy, fx)

# ff =

#    0.1711 - 0.2653i   0.2890 - 0.0884i   0.2890 + 0.0884i   0.1711 + 0.2653i
#    0.0532 - 0.0884i   0.1711 - 0.0295i   0.1711 + 0.0295i   0.0532 + 0.0884i
#    0.0532 + 0.0884i   0.1711 + 0.0295i   0.1711 - 0.0295i   0.0532 - 0.0884i
#    0.1711 + 0.2653i   0.2890 + 0.0884i   0.2890 - 0.0884i   0.1711 - 0.2653i


xx = np.linspace(-1,1,4)
xx = np.vstack((xx,xx,xx,xx))
yy = np.transpose(xx)


xcheb, x1, x2, x3 = chebpts(4)
f = 1/(1+ 25*xcheb**2)

baryCheck = bary(xx+1j*yy,f)
print(baryCheck)

[[0.17108753-0.26525199j 0.28897731-0.08841733j 0.28897731+0.08841733j
  0.17108753+0.26525199j]
 [0.05319776-0.08841733j 0.17108753-0.02947244j 0.17108753+0.02947244j
  0.05319776+0.08841733j]
 [0.05319776+0.08841733j 0.17108753+0.02947244j 0.17108753-0.02947244j
  0.05319776-0.08841733j]
 [0.17108753+0.26525199j 0.28897731+0.08841733j 0.28897731-0.08841733j
  0.17108753-0.26525199j]]


In [19]:
#=====Check cumsummat=====

# checkCumSumat = chebcolloc2.cumsummat(4)

# checkCumSumat =

#          0         0         0         0
#     0.1736    0.3889   -0.1111    0.0486
#     0.0625    1.0000    0.5000   -0.0625
#     0.1111    0.8889    0.8889    0.1111

checkCumSummat = cumsummat(4)
print(checkCumSummat)

[[ 0.          0.          0.          0.        ]
 [ 0.17361111  0.38888889 -0.11111111  0.04861111]
 [ 0.0625      1.          0.5        -0.0625    ]
 [ 0.11111111  0.88888889  0.88888889  0.11111111]]


In [148]:
#=====Check Asy2======

#VERIFIED

# x
# 0.176101651563189
# 0.410618524152389
# 0.620838010753476
# 0.794568599002320
# 0.921819868228935

# w
# 0.00664866161087821	0.00544241040737603	0.00294985525077371	0.000921144105071309	0.000109382815142530

# v
# 0.0802650278890440
# -0.0672664760538901
# 0.0425777351192206
# -0.0184279588881962
# 0.00405395652273077

asy2Testx, asy2Testw, asy2Testv = asy2_1(10,2,3,5)
print('x')
print(asy2Testx)
print('w')
print(asy2Testw)
print('v')
print(asy2Testv)

x
[[0.17610165]
 [0.41061853]
 [0.62083802]
 [0.79456861]
 [0.92181987]]
w
[[0.0066487  0.00544245 0.00294987 0.00092115 0.00010938]]
v
[[ 0.08026528]
 [-0.06726669]
 [ 0.04257787]
 [-0.01842802]
 [ 0.00405397]]


In [136]:
#=====Check Bessel Taylor=====

#VERIFIED
# 0.124348547436763
# -0.156641558376042
# 0.175029726558271
# -0.183726986614178
# 0.184483982407096

tTest = np.array([0.3980, 0.6524, 0.9009, 1.1475, 1.3935])
tTest = tTest.reshape(-1,1)
besselTaylorTest = besselTaylor(-tTest,13*tTest,2)
print(besselTaylorTest)

[[ 0.12436746]
 [-0.15671975]
 [ 0.1749767 ]
 [-0.18373493]
 [ 0.18452128]]


In [129]:
#=====Check Asy2 Higher Terms=====

#Returns function handles, output is irrelevant

testAsy2HigherTerms = asy2_higherterms(2,3,
                                       [0.398004552458513, 0.652426404542027, 0.900874970697452, 1.14750554324891, 1.39354401687791],
                                       10)
#print(testAsy2HigherTerms)

In [139]:
#=====CHECK FEVAL ASY2=====

# vals
# 0.00403562919865285
# -0.00241170329723305
# 0.00202810089792115
# -0.00214842948573832
# 0.00279022904227691

# ders
# 95.6620349801267
# -32.9634940768811
# 18.4189770231067
# -13.5585736482667
# 12.2641326212076


tTest = np.array([0.3980, 0.6524, 0.9009, 1.1475, 1.3935])
tTest = tTest.reshape(-1,1)
fevalAsy2Test1, fevalAsy2Test2 = feval_asy2(10, tTest, 1, 2, 3, 13)
print("vals")
print(fevalAsy2Test1)
print("ders")
print(fevalAsy2Test2)

[[0.398 ]
 [0.6524]
 [0.9009]
 [1.1475]
 [1.3935]]
vals
[[ 0.0044721 ]
 [-0.00328244]
 [ 0.00156722]
 [-0.00222364]
 [ 0.00333006]]
ders
[[ 95.66736452]
 [-32.9689635 ]
 [ 18.41742467]
 [-13.5587239 ]
 [ 12.26417104]]
